## Prologエンジンのpythonへの移植
Steven John Metsker著「<a href="https://www.amazon.com/Building-Parsers-Java%C2%BF-Steven-Metsker/dp/0201719622">JAVAによるサーバ構築技法（Building Parsers With Java）</a>」に付属している推論エンジンとパーサをpythonに移植しました。
同ソースコードは、以下のURLで公開されています。
- https://github.com/sujitpal/bpwj

CDROMのreadme.txtによるとコードは自由に使ってよいと言うことでこれをベースにpythonにエンジンを移植していきます。
```
Copyright
---------
The code on the CD is free. It is copyrighted, so you may not claim that 
you wrote the code. Otherwise you may use the code as you wish.
```

エンジンのポーティングは、「JAVAによるサーバ構築技法」の12章をベースに行いました。
この章の例題を移植したソースに適応して、動作を確認しながらポーティング作業を進めました。

### 移植のポイント
エンジンのメイン機能は、単一化（Unification）とバックトラック処理にあります。

論理エンジンは、Structureクラスをベースとして作られています。
構造（Structure）は、関数子（functor）と項（terms）の配列で構成されています。

項を持たず、関数子だけをもつ構造をアトム（Atom）と呼びます。

エンジンの最終的なポート結果は、engine.pyにあります。

## Unificationの移植
Javaのソースは、クラス毎に定義され、相互参照をしているため、Pythonで一つのファイルで定義するには、そのクラスがグローバルに定義されているものとしてコーディングする必要があります。

最初に必要なライブラリをインポートします。

```python
import copy
from collections import defaultdict
import time
```

次にUnificationを移植します。bpwj/src/main/java/sjm/engine/にエンジン周りのJavaクラスが定義されています。

Unification.javaをPythonに移植します。
Javaソースのvectorをself._variablesとして実装しました。

Unificationクラスは、変数（Variable）の単一化の結果を_variablesに保持します。


Unificationクラスの移植では、クラス変数emptyの初期化をクラスインスタンス生成メソッド__new__の中で実装することです。

```python
class Unification():
    empty = None
    def __new__(cls, *args, **kargs):

        if cls.empty == None:
            cls.empty = super(Unification, cls).__new__(cls)
            cls.empty._variables = []
        return super(Unification, cls).__new__(cls)

    def __init__(self, v = None):
        self._variables = []
        if v != None:
            self.addVariable(v)
```


## Structureの移植
Sturctureクラスのコンストラクターは、以下のようになります。

```python
class Structure:
    # コンストラクター
    def __init__(self, functor, terms = []):
        self.functor = functor
        self.terms = terms
        if terms != None and len(terms) > 0:
            self.terms = terms
```

これをjavaの定義と比較すると、javaでは複数のコンストラクターが定義できるのに対して、pythonでは__init__の１つしか定義できません。この点がpythoへの移植の厄介なところでした。
```java
public class Structure implements Term {
	protected Object functor;
	protected Term[] terms;

    public Structure(Object functor) {
        this(functor, new Term[0]);
    }
    public Structure(Object functor, Term[] terms) {
        this.functor = functor;
        this.terms = terms;
    }    
}
```


engine.pyからStructureクラスを抜粋します（コメントを除く）。

ここで注目して欲しいのは、以下の３つです。
- コンストラクター（__init__）
- 単一化（unify）
- 同一判定（__eq__）

それと__str__は、インスタンスの表示の時に役に立つので、最初に実装するのがポイントです。また、一度にすべてのメソッドを実装するのではなく、最低限のメソッドから実装し、動作を確認しながら薦めると良いでしょう。

```python
class Structure:
    # 以下抜粋
    # コンストラクター
    def __init__(self, functor, terms = []):      
        self.functor = functor
        self.terms = terms
        if terms != None and len(terms) > 0:
            self.terms = terms
    # 単一化
    def unify(self, s):
        if isinstance(s, Structure):
            if not self.functorAndArityEquals(s):
                return None
            u = Unification()
            others = s.terms
            for i in range(len(self.terms)):
                subUnification = self.terms[i].unify(others[i])            
                if subUnification == None:
                    u.unbind()
                    return None
                u.append(subUnification)
            return u
        elif isinstance(s, Variable):
            v = s
            return v.unify(self)
        else: # Term
            t = s
            return t.unify(self)
    # 同一判定
    def __eq__(self, o):       
        if type(self) != type(o):
            return False
        s = o
        if not self.functorAndArityEquals(s):
            return False
        for i in range(self.arity()):
            if not self.terms[i].__eq__(s.terms[i]):
                return False
        return True
```

### 動作確認
「JAVAによるサーバ構築技法」のすぐれたところは、クラスの定義と動作確認が実装されていることです。
- src/main/java/sjm/examples/に動作確認用のクラスが定義されています。

ShowStructure.javaをPythonに移植して動作を確認します。

Structureを使ったAtomの例として、denver（デンバー）を定義すると以下のようになります。

In [12]:
from engine import *

denver = Structure("denver")
print(denver)

denver


構造を使ってcityの地名（Name）と標高（altitude）を定義すると、以下のようになります。

In [13]:
denver = Structure("denver")
alititude = Structure(5280)
city = Structure("city", [denver, alititude])
print(city)

city(denver, 5280)


## 変数(Variable)の移植

次に変数（Variable）を移植します。

ここでのポイントは以下のメソッドです。
- コンストラクター（__init__）
- 単一化（unify）
- 単一化の解除（unbind）



```python
class Variable: 
    # 抜粋
    def __init__(self, name):    
        self.name = name
        self.instantiation = None
        self.id = f"{name}_{str(time.time())}"

    def unify(self, s):     
        structureCls = globals()['Structure']
        if isinstance(s, Variable):
            v = s
            if self is v:
                return Unification()
            elif self.instantiation != None:
                return self.instantiation.unify(v)
            elif v.instantiation != None:
                return v.instantiation.unify(self)
            self.instantiation = v
            return Unification(self)
        elif isinstance(s, structureCls):
            if self.instantiation != None:
                return self.instantiation.unify(s)
            self.instantiation = s
            return Unification(self)
        else: # Term
            t = s
            return t.unify(self)
        
    def __eq__(self, o):      
        if not isinstance(o, Variable):
            return False
        v = o
        if self.name != v.name:
            return False
        if self.instantiation == None:
            return v.instantiation == None
        return self.instantiation.__eq__(v.instantiation)
```

### 単一化（Unification）
変数（Variable）の単一化の結果を保持するクラスがUnificationです。

JAVAによるサーバ構築技法から単一化の方法をまとめると以下の通りです。
- ２つの構造が単一化できるためには、両者が同じ関数子を持ち、項の数が等しく、
対応する項同士が単一かできる必要がある
- インスタンス化されていない変数を特定の構造に単一化すると、その構造が変数の値になる
- インスタンス化された変数は、インスタンス値に単一化を依頼することで、自身の単一化を実行する

先に作成した構造体cityと変数name, altの単一化をすると、各変数のinstance属性に構造体denverとalititudeがセットされていることを確かめてみましょう(ShowStructureUnification.javaの移植)。

In [14]:
from engine import *

# Variableの確認
name = Variable("Name")
alt = Variable("Altitude")
vCity = Structure("city", [name, alt])
print(vCity)

city(Name, Altitude)


unifyメソッドで変数name, altにvCityの値（denver, 5280）が単一化されます。

In [15]:
vCity.unify(city)
print(f"name={str(name)}, alt={str(alt)}")

name=denver, alt=5280


In [16]:
# 以下のコマンドをデバッガで実行すると、変数name, altに
# セットされているinstatiationに構造cityのdenver, alititude
# であることが確認できます。
print(name.instantiation)
print(alt.instantiation)

denver
5280


デバッガ画面で、name.instantiationに構造(0x110e51930)がセットされ、それが構造(denver)であり、そのfunctorの値がdenverであることが確認できます。

<img src="images/unification.png">

## Factの移植
事実（Fact）は、変数を含まない構造（Struct）と説明されています。

Factには、様々なコンストラクターが用意されており、事実の記述が簡単にできるように配慮されています。

Javaのコンストラクターには、以下のメソッドが提供されています。

- Fact(functor: Object)
- Fact(functor: Object, term0: Object)
- Fact(functor: Object, term0: Object, term1: Object)
- Fact(functor: Object, terms[]: Object)
- Fact(functor: Object, terms[]: Fact)

これをpythonのコンストラクター（__init__）で以下のように定義しています。

```python
class Fact(Structure):    
    def __init__(self, functor, *args):     
        if len(args) == 0:
            super().__init__(functor)
        elif len(args) == 1:            
            if isinstance(args[0], list):
                first = args[0]
                if len(first) > 0 and isinstance(first[0], Fact):
                    super().__init__(functor, first)
                else:
                    atoms = [Atom(o) for o in first]
                    super().__init__(functor, atoms)
            elif isinstance(args[0], Fact):
                super().__init__(functor, [args[0]])
            else:
                super().__init__(functor, [Atom(args[0])])
        elif len(args) == 2:
            o1 = args[0]
            o2 = args[1]
            if isinstance(o1, Atom):
                super().__init__(functor, [o1, o2])
            else:
                super().__init__(functor, [Atom(o1), Atom(o2)])

```

### Factの動作確認
Factの動作確認用にShowFactで定義されている処理をPythonで実行してみます。

多様なコンストラクターの例となっています。

In [2]:
from engine import *

d = Fact("city", [Fact("denver"), Fact(5280)])
j = Fact("city", "jacksonville", 8)

print(d)
print(j)

city(denver, 5280)
city(jacksonville, 8)


## ProgramとQueryの移植


# prologの例題
Logikusパーサを使って「Prologの技芸（art of prolog）」の例題を試してみます。

ちなみにPrologの技芸は私の前職の「株式会社 構造計画研究所」が発行した書籍です。



In [3]:
from parser import *

## ３章　再帰プログラミング

### 自然数

### 自然数の計算
自然数の「和」をprologで定義します。

自然数の表現にはs(0)が0に１足したことを表すs表現を使用します。

In [4]:
s = """
natural_number(0);
natural_number(s(X)) :- natural_number(X);
plus(0, X, X) :- natural_number(X);
plus(s(X), Y, s(Z)) :- plus(X, Y, Z);
"""
p = LogikusFacade.program(s)
LogikusFacade.query('plus(s(s(s(0))), s(s(0)), Y);', p)

Y = s(s(s(s(s(0)))))
Yes


この結果は、s(s(s(0)))=3, s(s(0))=2なので、s(s(s(s(s(0)))))=5と読みます。

同様に「積」、「べき乗」を追加します。

In [5]:
s = """
natural_number(0);
natural_number(s(X)) :- natural_number(X);
plus(0, X, X) :- natural_number(X);
plus(s(X), Y, s(Z)) :- plus(X, Y, Z);
times(0, X, 0) :- natural_number(X);
times(s(X), Y, Z) :- times(X, Y, XY), plus(XY, Y, Z);
exp(s(N), 0, 0);
exp(0, s(X), s(0));
exp(s(N), X, Y) :- exp(N, X, Z), times(Z, X, Y);
"""
Program.debug = False
p = LogikusFacade.program(s)
LogikusFacade.query('exp(s(s(s(0))), s(s(0)), Yq);', p)

Yq = s(s(s(s(s(s(s(s(0))))))))
Yes


In [6]:
s = """
imul(_, [], []);
imul(C, [X|Xs], [Z|Zs]) :- #(Z, C * X), imul(C, Xs, Zs);
"""
p = LogikusFacade.program(s)
LogikusFacade.query('imul(2, [1,-1,0,2,-3], Vs);', p)

Vs = [2, -2, 0, 4, -6]
Yes


In [7]:
s = """
imul(_, [], []);
imul(C, [X|Xs], [Z|Zs]) :- #(Z, C * X), imul(C, Xs, Zs);

add([], Ys, Ys);
add([X|Xs], [], [X|Xs]);
add([X|Xs], [Y|Ys], [Z|Zs]) :-
	#(Z, X + Y),
	add(Xs, Ys, Zs);
"""
p = LogikusFacade.program(s)
LogikusFacade.query('add([1,2,3], [1,-2,3,-4], Vs)', p)

Vs = [2, 0, 6, -4]
Yes


In [8]:
s = """
imul(_, [], []);
imul(C, [X|Xs], [Z|Zs]) :- #(Z, C * X), imul(C, Xs, Zs);

add([], Ys, Ys);
add([X|Xs], [], [X|Xs]);
add([X|Xs], [Y|Ys], [Z|Zs]) :-
	#(Z, X + Y),
	add(Xs, Ys, Zs);

mul([], _, []);
mul([X|Xs], Ys, Zs) :-
	imul(X, Ys, Zs1),
	mul(Xs, Ys, Zs2),
	add(Zs1, [0|Zs2], Zs);
"""
p = LogikusFacade.program(s)
LogikusFacade.query('mul([1,2,3], [1,-2,3], Vs);', p)

Vs = [1, 0, 2, 0, 9]
Yes


In [9]:
s = """
append([], Z, Z) ;
append([W|X1], Y, [W|Z1]) :- append(X1, Y, Z1);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('append([1], [0], V);', p)


append([], Z, Z);
append([W|X1], Y, [W|Z1]) :- append(X1, Y, Z1);
V = [1, 0]
Yes


### ハノイの塔の例題

In [10]:
s = """
append([], Ys, Ys);
append([X|Xs], Ys, [X|Zs]) :- append(Xs, Ys, Zs);

// hanoi(s(0), A, B, C, [[A, to, B]]);
hanoi(0, A, B, C, []);
hanoi(s(N), A, B, C, Moves) :-
	hanoi(N, A, C, B, Ms1),
	hanoi(N, C, B, A, Ms2),
	append(Ms1, [[A, to, B]|Ms2], Moves);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('hanoi(s(s(s(0))), "A", "B", "C", Vs);', p)


append([], Ys, Ys);
append([X|Xs], Ys, [X|Zs]) :- append(Xs, Ys, Zs);
hanoi(0, A, B, C, []);
hanoi(s(N), A, B, C, Moves) :- hanoi(N, A, C, B, Ms1), hanoi(N, C, B, A, Ms2), append(Ms1, [[A, to, B]|Ms2], Moves);
Vs = [[A, to, B], [A, to, C], [B, to, C], [A, to, B], [C, to, A], [C, to, B], [A, to, B]]
Yes


In [11]:
s = """
city(denver, 5280);
city("jacksonville", 8);
city("test hello", 0);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('city(Name, Heigh);', p)

city(denver, 5280);
city(jacksonville, 8);
city("test hello", 0);
Name = denver, Heigh = 5280
Name = jacksonville, Heigh = 8
Name = "test hello", Heigh = 0
Yes


## 日本語も使える

In [12]:
s = """
"市"("高岡", 100);
"市"("富山", 20);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('"市"(Name, Heigh);', p)

"市"(高岡, 100);
"市"(富山, 20);
Name = 高岡, Heigh = 100
Name = 富山, Heigh = 20
Yes


## デバッグトレース

In [13]:

Program.debug = True
s = """
append([X|Xs], Ys, [X|Zs]) :- append(Xs, Ys, Zs);
append([], Ys, Ys);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('append(X, Y, [a, b, c]);', p)

Program.debug = False

append([X|Xs], Ys, [X|Zs]) :- append(Xs, Ys, Zs);
append([], Ys, Ys);
append(X, Y, [a, b, c])?
	append([X|Xs], Ys, [X|Zs])	True	X = [a|Xs], Ys = Y, X = a, Zs = [b, c] => append(Xs, Y, [b, c])
	append([X|Xs], Ys, [X|Zs])	True	Xs = [b|Xs], Y = Ys, X = b, Zs = [c] => append(Xs, Ys, [c])
	append([X|Xs], Ys, [X|Zs])	True	Xs = [c|Xs], Ys = Ys, X = c, Zs = [] => append(Xs, Ys, [])
	append([X|Xs], Ys, [X|Zs])	False
	append([], Ys, Ys)	True	Xs = [], Ys = [], Ys = []
	Return: Xs = [], Ys = [], Zs = []
	Return: Xs = [c], Ys = [], Zs = [c]
	Return: Xs = [b, c], Ys = [], Zs = [b, c]
	Return: X = [a, b, c], Y = []
X = [a, b, c], Y = []
	append([], Ys, Ys)	True	Xs = [], Ys = [c], Ys = [c]
	Return: Xs = [], Ys = [c], Zs = [c]
X = [a, b], Y = [c]
	append([], Ys, Ys)	True	Xs = [], Y = [b, c], Ys = [b, c]
	Return: Xs = [], Ys = [b, c], Zs = [b, c]
X = [a], Y = [b, c]
	append([], Ys, Ys)	True	X = [], Ys = [a, b, c], Y = [a, b, c]
	Return: X = [], Y = [a, b, c]
X = [], Y = [a, b, c]
Yes


## 9章

In [14]:
s = """
isEmpty([]);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('not isEmpty([a]);', p)

isEmpty([]);

Yes


In [15]:
s = """
append([X|Xs], Ys, [X|Zs]) :- append(Xs, Ys, Zs);
append([], Ys, Ys);
isEmpty([]);
flatten([X|Xs], Ys3) :- flatten(X, Ys1), flatten(Xs, Ys2), append(Ys1, Ys2, Ys3);
flatten(X, [X]) :- constant(X), not isEmpty(X);
flatten([], []);
"""
# flatten(Xs, Ys) :- Ysは、Xsの要素のリストである。
# constantが未定義なのでうまく動作しない
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('flatten([[a], [b, [c, d], e]], Y);', p)

append([X|Xs], Ys, [X|Zs]) :- append(Xs, Ys, Zs);
append([], Ys, Ys);
isEmpty([]);
flatten([X|Xs], Ys3) :- flatten(X, Ys1), flatten(Xs, Ys2), append(Ys1, Ys2, Ys3);
flatten(X, [X]) :- constant(X), not isEmpty(X);
flatten([], []);
No
